In [2]:
from utils import *

Source: https://archive.physionet.org/physiobank/database/html/mitdbdir/

# Selection criteria
The source of the ECGs included in the MIT-BIH Arrhythmia Database is a set of over 4000 long-term Holter recordings that were obtained by the Beth Israel Hospital Arrhythmia Laboratory between 1975 and 1979. Approximately 60% of these recordings were obtained from inpatients. The database contains 23 records (numbered from 100 to 124 inclusive with some numbers missing) chosen at random from this set, and 25 records (numbered from 200 to 234 inclusive, again with some numbers missing) selected from the same set to include a variety of rare but clinically important phenomena that would not be well-represented by a small random sample of Holter recordings. Each of the 48 records is slightly over 30 minutes long.

The first group is intended to serve as a representative sample of the variety of waveforms and artifact that an arrhythmia detector might encounter in routine clinical use. A table of random numbers was used to select tapes, and then to select half-hour segments of them. Segments selected in this way were excluded only if neither of the two ECG signals was of adequate quality for analysis by human experts.

Records in the second group were chosen to include complex ventricular, junctional, and supraventricular arrhythmias and conduction abnormalities. Several of these records were selected because features of the rhythm, QRS morphology variation, or signal quality may be expected to present significant difficulty to arrhythmia detectors; these records have gained considerable notoriety among database users.

The subjects were 25 men aged 32 to 89 years, and 22 women aged 23 to 89 years. (Records 201 and 202 came from the same male subject.)


# Data Exploration

In [3]:
# Load data from MIT-BIH Arrhythmia Database
# https://physionet.org/content/mitdb/1.0.0/

dict_signals = {}
list_annotations = []

files = glob(f'{mb_artm_directory}*dat')
for file in files:
    record_path = file.replace(".dat",'')
    record = wfdb.rdrecord(record_path)
    dict_signals[(record.record_name)] = record.sig_name
    ann = wfdb.rdann(record_path,'atr')
    list_annotations.append(pd.Series(ann.symbol).value_counts().to_dict())
columns = ['upper_signal', 'lower_signal']

# Dataframe with lead configurations for upper and lower signals for each record
df_record_lead = pd.DataFrame(dict_signals, index=columns).T.reset_index().rename(columns={'index':'record'})
df_record_lead.record = df_record_lead.record.astype(np.int32)
df_record_lead['group'] = 'random'
df_record_lead.loc[df_record_lead.record >= 200, 'group'] = 'selected'


# Dataframe with the number of configurations for upper and lower signals
df_record_lead_summery = df_record_lead.groupby(['group', 'upper_signal','lower_signal']).count().reset_index().sort_values('record', ascending=False, ignore_index=True)

# Dataframe with the number of annotations for each record
df_ann = pd.DataFrame(list_annotations).fillna(0)
df_ann = df_ann.astype(int)
df_ann.insert(0, 'record', df_record_lead.record)

# Display first values of dataframes
print("df_record_lead")
display(df_record_lead.head())

print("df_record_lead_summery")
display(df_record_lead_summery)

print("df_ann")
display(df_ann.head())

df_record_lead


,record,upper_signal,lower_signal,group
0,100,MLII,V5,random
1,101,MLII,V1,random
2,102,V5,V2,random
3,103,MLII,V2,random
4,104,V5,V2,random


df_record_lead_summery


,group,upper_signal,lower_signal,record
0,selected,MLII,V1,25
1,random,MLII,V1,15
2,random,MLII,V2,2
3,random,MLII,V5,2
4,random,V5,V2,2
5,random,MLII,V4,1
6,random,V5,MLII,1


df_ann


,record,N,A,+,V,~,|,Q,/,f,x,F,j,L,a,J,R,!,E,[,],S,"""",e
0,100,2239,33,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,101,1860,3,1,0,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,102,99,0,5,4,0,0,0,2028,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,103,2082,2,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,104,163,0,45,2,37,0,18,1380,666,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# Dataframe with description of each annotation code
df_code_description = pd.concat(pd.read_html("https://archive.physionet.org/physiobank/annotations.shtml")[:2])[['Code', 'Description']].dropna().reset_index(drop=True)

# Display first values of dataframe
print("df_code_description")
df_code_description.head()

df_code_description


,Code,Description
0,N,"Normal beat (displayed as ""·"" by the PhysioBan..."
1,L,Left bundle branch block beat
2,R,Right bundle branch block beat
3,B,Bundle branch block beat (unspecified)
4,A,Atrial premature beat


In [5]:
series_beat_codes = df_code_description.Code.iloc[:19]
print(series_beat_codes)

0     N
1     L
2     R
3     B
4     A
5     a
6     J
7     S
8     V
9     r
10    F
11    e
12    j
13    n
14    E
15    /
16    f
17    Q
18    ?
Name: Code, dtype: object


In [6]:
# Dataframe with the number of annotations for each code and the respective description
df_ann_summery =  df_ann[df_ann.columns.to_list()[:-1]].sum(axis=0).reset_index().rename(columns={'index':'Code', 0:'Count'}).merge(df_code_description, on='Code').sort_values('Count', ascending=False).reset_index(drop=True)

# Display dataframe
df_ann_summery

,Code,Count,Description
0,N,75052,"Normal beat (displayed as ""·"" by the PhysioBan..."
1,L,8075,Left bundle branch block beat
2,R,7259,Right bundle branch block beat
3,V,7130,Premature ventricular contraction
4,/,7028,Paced beat
5,A,2546,Atrial premature beat
6,+,1291,Rhythm change [2]
7,f,982,Fusion of paced and normal beat
8,F,803,Fusion of ventricular and normal beat
9,~,616,Change in signal quality [1]


In [7]:
# Create a more comprehensive summery with number of annotations for each code for each lead configuration 

df_record_lead_ann = df_record_lead.merge(df_ann, on='record')


df_lead_ann_summery = df_record_lead_ann.groupby(['group','upper_signal','lower_signal'])[df_ann_summery.Code[:-1]].sum().reset_index().sort_values('N', ascending=False, ignore_index= True)

# Display first values of dataframe

print("df_record_lead_ann")
display(df_record_lead_ann.head())

print("df_lead_ann_summery")
display(df_lead_ann_summery)

df_record_lead_ann


,record,upper_signal,lower_signal,group,N,A,+,V,~,|,Q,/,f,x,F,j,L,a,J,R,!,E,[,],S,"""",e
0,100,MLII,V5,random,2239,33,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,101,MLII,V1,random,1860,3,1,0,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,102,V5,V2,random,99,0,5,4,0,0,0,2028,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,103,MLII,V2,random,2082,2,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,104,V5,V2,random,163,0,45,2,37,0,18,1380,666,0,0,0,0,0,0,0,0,0,0,0,0,0,0


df_lead_ann_summery


,group,upper_signal,lower_signal,N,L,R,V,/,A,+,f,F,~,!,"""",j,x,a,|,E,J,Q,[,]
0,selected,MLII,V1,43507,3460,3562,5784,1542,2391,1064,260,790,338,472,437,223,172,144,81,106,52,8,6,6
1,random,MLII,V1,22093,4615,2166,1246,2078,107,157,0,4,223,0,0,1,21,6,50,0,0,7,0,0
2,random,MLII,V5,3754,0,0,4,0,33,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,random,MLII,V2,3616,0,0,0,0,3,2,0,0,9,0,0,0,0,0,0,0,0,0,0,0
4,random,V5,MLII,1820,0,0,43,0,10,3,0,4,7,0,0,0,0,0,1,0,2,0,0,0
5,random,V5,V2,262,0,0,6,3408,0,50,722,0,37,0,0,0,0,0,0,0,0,18,0,0
6,random,MLII,V4,0,0,1531,47,0,2,13,0,5,2,0,0,5,0,0,0,0,29,0,0,0


In [8]:
df_record_lead_ann.to_parquet(join(dataframes_directory, 'df_record_lead_ann.parquet'))
df_lead_ann_summery.to_parquet(join(dataframes_directory, 'df_lead_ann_summery.parquet'))

# Conclusions

- The majority of the records use the MLII-V1 lead configuration (40/48)
- In the random selected records, 15 out of 23 use the MLII-V1 lead configuration
- In the specially selected records, all 25 records use the MLII-V1 lead configuration